# Introduction to arithmetic coding

This notebook will recall on the fundamentals of information theory and then move to introduce the main principles and properties of arithmetic coding.

## Recall on fundamentals of information theory
Coding is all about finding a representation for the data we consider conveying information (e.g. velocities measured by a speed sensor, intensity of image pixels, etc.). Roughly speaking we have two types of coding:
 * *Fixed Length Coding* (FLC), whereby the most common example is the binary representation of a number.
 * *Variable Length Coding* (VLC), whereby the main idea is to assign short representations (i.e. codewords) to information values which appear more frequently. Examples of VLC are Huffman, run length and arithmetic coding.

## The advantage of variable length coding
Let's consider the example of coding a quote from a movie where, without loss of generality, we encode only the alphabetic characters associated with the letters in the English alphabet (i.e. 26 letters).

Our quote from the Silence of the Lambs, (by J. Demme, 1991) is:

<font size="4">**Hannibal Lecter**: *You're so ambitious, aren't you? You know what you look like to me, with your good bag and your cheap shoes? You look like a rube. A well-scrubbed, hustling rube, with a little taste. Good nutrition has given you some length of bone, but you're not more than one generation from poor white trash, are you, Agent Starling? And that accent you've tried so desperately to shed? Pure West Virginia. What'd your daddy do, was he a coal miner? Does he stink of the lamp? Oh and how quickly the boys found you ... all those sticky, tedious fumblings in the back seats of cars ... while you could only dream of getting out ... getting anywhere ... getting all the way to the FBI.*</font>


In [ ]:
with open('dr-lecter-meets-starling.txt', 'rt') as fh:
    quote = fh.read()
quote = quote.lower()
total_alphabetic_characters = sum(c.isalpha() for c in quote)
total_morse = sum(c in ('e', 't', 'a', 'i', 'n', 'o', 's', 'h') for c in quote)
total_rotw = total_alphabetic_characters - total_morse
print(f"Quote contains {total_alphabetic_characters} letters, FLC would require {total_alphabetic_characters * 5} bits")
print(f"Morse's most frequent letters account for {100 * total_morse / total_alphabetic_characters:.2f}%, remaining ones account for: {100 * total_rotw / total_alphabetic_characters:.2f}%")
print(f"VLC (4 bits for Morse's letters and 6 bits for others) would require {total_morse * 4 + total_rotw * 6} bits")

By executing the Python code above, we can see that a simple variable length coding scheme can help us in reducing the number of bits. Two questions may arise from this result:
 1. Is there a way to calculate the minimum amount of bits required to encode a given message?
 1. Is there a method that encodes with such a minimum amount of bits?
 
Let's start by addressing the first question: this requires to brush off some easy and fundamental results of the Shannon's information theory.

## Source of information and Shannon's entropy
The messages we want to encode can be thought as emitted by a source of information. Examples of sources of information are:
 * **Source** emitting integer numbers representing the speed of vehicles (in km/h) on Euston Rd. in central London. **Messages** are represented by sequences of these speeds.
 * **Source** emitting the dot and line characters representing the letters translated in the Morse code. **Messages** are represented by sequences of dot and line characters.
 * **Source** emitting integer numbers representing the pixel intensity associated with gray scale images portraying natural content. **Messages** are represented by gray scale images of indoor and/or outdoor scenes.

Mathematically we can describe a source of information $S$ as the resemble:
$$
\large
S = <A, p_S(\cdot)>,
$$

where $A$ is the source's alphabet, i.e. the set of admissible symbols (e.g. integer numbers) and $p_S(\cdot)$ is the Probability Mass Function (PMF)(\*), a function which maps each symbols $s_i \in S$ to the interval $[0, 1]$:

$$
\large
p_S(\cdot): A \rightarrow [0,1]
$$

(\*) Sometimes the PMF is confused with the Probability Density Function (PDF) but they're not the same. PDF refers to continues variables and requires integration to know the probability associated with a given value, whilst the PMF does not.

One example of source of information can be a binary source $B$ which emits the symbols *head* and *tail* with probability ${0.25, 0.75}$, respectively:

$$
\large
B=<\{head,\mbox{ }tail\}, \{0.25, 0.75\}>
$$

Over a source's symbol $s_i \in S$ we define its **information content** $I(s_i)$ as:
$$
\large
I(s_i) = \log_2\left(\frac{1}{p_S(s_i)}\right),\quad[bits]
$$

where $p_S(s_i)$ is the PMF's value associated with $s_i$. We observe the following:
 * Symbols which appear less frequently (i.e. having a small PMF value) will bring a large amount of information, many bits to encode them.
 * Conversely, symbols which appear more regularly do not bring much information, few bits to encode them.

We are now ready to introduce a fundamental concept which will allow us to answer the first question above: the **Shannon's entropy**. Given a source of information $S$, its entropy $H(S)$ is defined as the expected information content:

$$
\large
H(S) = E[I(s_i)] = \sum_{i=0}^N -p_S(s_i)\cdot\log_2(p_S(s_i))\quad[bits/symbol]
$$

Few considerations from this definition:
 * The entropy is easy to compute, the PMF, $p_S(\cdot)$ is (usually) difficult to obtain.
 * The entropy is bounded below by zero.
 * The entropy represents the ***expected*** number of bits one would need to encode the messages emitted by $S$.
 * The entropy is equal to $\log_2(|A|)$ if an only if $p_S(\cdot)$ is uniform (with $|\cdot|$ denoting the set's cardinality).

## Addendum: Shannon's entropy of a binary source
We'll study here the entropy for a simple case of binary source $B$, i.e. a source of information which emits only two symbols. Suppose that our source emits symbol $1$ with probability $p$ and (hence) symbol $0$ with probability $1-p$. The entropy is given as:

$$
\large
H(B) = -p\cdot \log_2(p) -(1-p)\cdot \log_2(1-p) = p\cdot\log_2\left(\frac{1-p}{p}\right) - \log_2(1-p)
$$

The cell below plots $H(B)$ as a function of $p$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def expected_info(a):
    idx_valid = np.where(a)
    idx_zero = np.where(a == 0)
    result = np.zeros(a.shape)
    result[idx_valid] = -np.multiply(a[idx_valid], np.log2(a[idx_valid]))
    return result

p = np.linspace(0, 1, 100)
hb = expected_info(p) + expected_info(1-p)
plt.plot(p, hb, 'b')
plt.grid()
plt.xlabel('$p$', fontsize=16)
plt.ylabel('$H(B)$ [bits/symbol]', fontsize=16);

We note that when $p = 0.5$ the entropy is equal to $1$ bit per symbol, i.e. VLC will not provide any benefit. We also note that when $p = 0$ and $p = 1$ the entropy is equal to zero, despite the formula would have the logarithm terms undefined. In fact, these two values imply that the source will always emit symbol $0$ or symbol $1$  (respectively) and there is no point in applying any entropy coding since the outcome of the source is known a priori.

Re-considering the example of the movie's quote above we would like to understand whether the encoding used (i.e. 3 bits for Morse's most frequent letters and 5 bits for the others) is efficient. Assume that our movie's quote was produced by a source which emits Morse's letters with probability 0.6 and non Morse's letters equal to 0.4. For every message, our encoding scheme will use on ***average***:

$$
\bar{L} = 0.6 \cdot 4 + 0.4 \cdot 6 = 4.8 \quad[bits],
$$

where $\bar{L}$ is the ***average codeword length***. The entropy for this source is given as (\*):

$$
H(quote) = 4.18 \quad[bits]
$$

(\*): See the addendum cell below for an explanation on how this entropy was computed.

## Addendum: Few remarks on the entropy calculation for the movie's quote
The calculation of the entropy associated with the source emitting movies' quotes has been computed by approximating its PMF from the quote itself (given that we don't have any other alternative). Accordingly, the following Python code cell shows a procedure to compute it.

In [ ]:
from collections import Counter
counts = Counter(quote)

pmf = []
for l in counts:
    if l.isalpha():
        pmf.append(counts[l])

pmf = np.array(pmf)
total = np.sum(pmf)
pmf = pmf / total

h = -np.sum(np.multiply(pmf, np.log2(pmf)))
print(f"Entropy associated with the source emitting movies' quotes: {h:.3} [bits]")

It is not fortuitous that the entropy results in less average bits than our simple coding scheme. Indeed Shannon didn't limit his research in just defining the concept of entropy but he formulated the:
<h3><center>Shannon's noiseless source coding theorem</center></h3>

**Given a source of information** $S$**, for any coding scheme** $C$ **we have the following**:

$$
\large
\bar{L}_C \geq H(S)
$$

In other words, no matter how efficient is our coding scheme $C$, its average codeword length $\bar{L}_C$ will always be greater or equal to Shannon's source entropy.

This fundamental result allows us to answer the first question we posed before: in fact the **minimum amount of bits required to transmit the messages** emitted by a given source of information **is given by its entropy**.

Few remarks on this result:
 * The coding scheme can approach the Shannon's entropy if certain conditions are met (e.g. source's PMF for Huffman coding).
 * Be careful on which entropy you use to assess the efficiency of your coding scheme (e.g. entropy associated with the image pixels whilst the coding scheme works on transform and quantised coefficients).

We're now in a position to answer/address the second question: "*Is there a method that encodes with such a minimum amount of bits?*". The short answer is **yes** and it is called ***arithmetic coding***. The main idea behind arithmetic coding is not difficult to explain, sadly, to make it work in practice one needs to design properly the coding engine (and probably scratch their own head few times).

# The genesis of arithmetic coding: Elias coding
This type of coding is due to the American mathematician and engineer [Peter Elias](http://news.mit.edu/2001/elias). The main idea is to map a $N$-length sequence of symbols $s_{0:N}$ emitted by a source of information $S$ into an interval $I$:

$$
\large
I = [low, high) \in [0, 1) \wedge low \leq high.
$$

The method is **designed to provide the interval's length equal to** $p_S(s_{0:N})$ (i.e., $p_S(s_{0:N}) = high - low$). You may wonder why this design requirement. Hold tight, hopefully it'll be clear in a moment but for the time being bear in mind that, according to the definition of information content, the quantity $-\log_2(p_S(s_{0:N}))$ will give us the minimum number of bits required to transmit $s_{0:N}$.

Elias coding works with any arity of the source (i.e. ternary, quaternary sources, etc.) but hereafter we'll just consider binary sources with PMF:

$$
\large
p_S : \{0, 1\} \rightarrow \{1 - p, p\}
$$

We'll also restrict our analysis to the so-called iid (identically and independently distributed) sources, that is sources whereby the emission of a symbol at any time doesn't depend on what the source has emitted in the past. Accordingly, the probability associated with a given $s_{0:N}$ is given by:

$$
\large
p_S(s_{0:N}) = (1 - p)^{n_0}\cdot p^{n_1}
$$

Elias coding takes a sequence of symbols as input and provides a unique codeword as output. This is different from block coding-based algorithms whereby each symbol is assigned to a specific codeword (e.g. Golomb codes).

How do we determine the final interval $I$? We subdivide the initial interval $I_0 = [0, 1)$ so that for each symbol $s_k$ a new interval $I_k = [low_k, high_k)$ is given as:

$$
\large
low_k = 
\begin{cases} 
low_{k-1} & s_k = 1\\ 
low_{k-1} + p\cdot(high_{k-1} - low_{k-1}) & s_k = 0
\end{cases}
$$
$$
\large
high_k = 
\begin{cases} 
low_{k-1} + p\cdot(high_{k-1} - low_{k-1}) & s_k = 1\\ 
high_{s-1} & s_k = 0
\end{cases}
$$

Let's see how the initial interval $I_0$ gets partitioned to $I_1$ depending on whether we're encoding a zero or one. The image below shows such partitioning.

<img src="elias-coding-1st-example.png" width="900">

## Another example
Let's consider now a slightly longer example whereby our binary source has $p(0) = 0.25$, $p(1) = 0.75$ and the message if a 4-length sequence $s_{0:3} = 0110$. The figure below summarises the extrema for the intervals $I_0, I_1, I_2, I_3, I_4$.

<img src="elias-coding-2nd-example.png" width="900">

We note from the drawing above that any number in the interval $I_4 = [0.85547, 0.89062]$ can be used as **codeword**. The decoder will therefore mimic the same behaviour of the encoder. In fact, let's suppose to send the floating point number $0.87305$:
 * The decoder knows this number is greater than $p = 0.75$ so a symbol $0$ must be emitted.
 * The interval $[0.75, 1]$ is then partitioned and the decoder this time knows that $0.87305$ is less $15/16$ (the upper bound for the interval associated with symbol $1$ in this iteration). Accordingly, a symbol $1$ must be emitted.
 * The process continues until all message's symbols have been emitted (note that the decoder needs to know the message's length).

The intervals $I$ found by Elias coding have the following properties:
 1. All intervals $I_n$, corresponding to all $n$-length messages $s_{0:n}$ emitted by the source $S$ are disjoint and the union of all these intervals gives $[0, 1)$.
 1. For each $n_1$, $n_2$ such that $n_2 > n_1$, with $s_{0:n_1}$ being a prefix for $s_{0:n_2}$, we have:

$$
\large
[low_{n_2}, high_{n_2}) \subseteq [low_{n_1}, high_{n_1})
$$

 3. The length of the interval associated with $s_{0:n}$ is given by:
 
$$
\large
high_n - low_n = \prod_{n=0}^{n-1} p_S(s_i) = p_S(s_{0:n})
$$

The last property is exactly what we said Elias coding is aiming for, i.e. the message $s_{0:n}$ is mapped into an interval whose length is equal to the probability of the message. And given that $-\log_2(p_S(s_{0:n}))$ gives the minimum number of bits required to transmit $s_{n:0}$, we now understand why Elias coding can guarantee to achieve the Shannon's entropy even if the source of information $S$ does not meet certain requirements (cmp. with the optimality condition required by Huffman coding, i.e. the source's PMF should have values which are negative powers of 2).

## To summarise
Our *codeword*, i.e. the fractional number lying in the final interval $I_N$, will be sent as a binary fraction:

$$
\large
codeword = 0.\underbrace{bbbb\ldots b}_{l}\quad\text{with } b\in\{0, 1\},
$$

where $l$ is ***any*** integer number such that $2^{-l} < high_n - low_n$. Accordingly we will also have:

$$
\large
l = -\log_2(high_n - low_n) = -\log_2(p_S(s_{0:n})) = \sum_{i=0}^{n-1}-\log_2(p_S(s_i))
$$

The average number of bits required to encode all possible $n$ symbols emitted by the source is:

$$
\large
E\left[-\log_2(p_S(s_{0:n}))\right] = E\left[\sum_{i=0}^{n-1}-\log_2(p_S(s_i))\right] = \sum_{i=0}^{n-1}E\left[-\log_2(p_S(s_i))\right] = n\cdot H(S).
$$

And this is another way to see how Elias coding achieves the Shannon's entropy.

Finally, let's focus again on the codeword to send. We said that any integer number in the interval $high_n - low_n$ is a valid codeword and it will require at *at least* $\lceil-\log_2(p_S(s_{0:n}))\rceil$ bits. So the question is: what's the codewords that spends the least bits? Let's consider a codeword with $l + 1$ bits. This corresponds:

$$
\large
codeword = 0.\underbrace{bb\ldots bbb}_{l}\bar{b} = \frac{1}{2}(high_n - low_n).
$$

Thus appending one bit to the binary fractional representation of the length of the interval associated with $s_{0:n}$ corresponds to the mid point of such a interval and guarantees that the decoder will be able to decode the message (since $2^{-(l+1)} < high_n - low_n$ is definitely satisfied).

## Play time
The Python code below implements Elias coding of a binary idd source emitting $0$ and $1$. You're requested to play with it by changing the length of the input message, varying the probability $p$, printing out some additional variables, etc. with the goal of explaining why the implementation below is not practical.

In [ ]:
# Example of Python code which implements Elias coding
import numpy as np
p = 0.75
message = '0110'
low_k, high_k = 0, 1

for m in message:
    if m == '1':
        high_k = low_k + p * (high_k - low_k)
    else:
        low_k = low_k + p * (high_k - low_k)
    k += 1

# Final calculations and print out
total_bits = np.ceil(-np.log2(high_k - low_k))
codeword = (high_k + low_k) / 2
bps = (total_bits + 1) / len(message)
print(f"Total bits: {total_bits:.2f}, bps: {bps:.2f}, final codeword: {codeword}")

In [ ]:
# Decoder
decoded_message = ''
current_codeword = codeword
for _ in range(len(message)):
    if 0 <= current_codeword and current_codeword < p:
        decoded_message += '1'
        current_codeword /= p
    else:
        decoded_message += '0'
        current_codeword = (current_codeword - p) / (1 - p)

print(f"Decoded message: {decoded_message}, decoding was {'successful' if decoded_message == message else 'unsuccessful'}")

## Solution
One way to prove why Elias coding is unpractical could consist in running the encoder over a longer message. As an example, by appending a few tens of zeros (say `011000000000000000000000000000`) and printing out the value for `high_k - low_k` at each iteration, we should observe that the value of the interval's length quickly reaches values in the order of $10^{-16}$ from the 29th iteration. Given that in practical schemes messages are usually way longer than 30 characters (as in this case), one can easily realise that the arithmetic precision required by Elias coding would quickly exceed even the one of modern and beefy machines. How to make Elias coding practical will be the topic of the second notebook under this tutorial.
